# How to Setup a Schedule with Pipeline Parameter Expressions
In this notebook, you will learn about pipeline parameter expressions, and how to set up a pipeline schedule with them.

## Introduction of pipeline parameter expression

Pipeline parameter allows user dynamically running pipeline experiments with different inputs and parameters without any code change. Compared to manual modification of parameter values each run, pipeline parameter expression can work as a more advanced mode by getting their values in runtime, which can relieve pain points in both regular run scheduling and pipeline iterative development. 

You could use invocation identifier `@` at the beginning of expression, and the expression value will be resolved in runtime.
For pipeline parameter expression containing nested functions like `@formatDateTime(subtractFromTime(utcNow(),1,'Day'),'yyyy-MM-dd')`, use only one `@` at the beginning of the expression, there is no need to add identifier to each of the functions.

See below table for supported expressions in schedule. The functions are case sensitive. Suffix `?` means optional parameter, otherwise required.

| Function    | Description |
| :---------- | :---------- |
| `trigger().scheduledTime` | Return time at which the trigger was scheduled to invoke the pipeline run. Consistent with the time you specific in `ScheduleRecurrence` instance. **This expression is only specific in schedule scenario.**
| `trigger().startTime` | Return time at which the trigger actually fired to invoke the pipeline run. This may be slightly later than the trigger's scheduled time. **This expression is only specific in schedule scenario.**
| `createdBy()` | Return a user's full name or a service principal's app ID as a string.
| `runId()` | Return run ID as a string of the top-level pipeline run. For nested subgraph runs, this is the run ID of the top-level pipeline run.  For non-nested runs, this is equal to pipeline run ID.
| `addDays('<timestamp>', <days>, '<format>'?)`    | Add a number of days to a timestamp.
| `addHours('<timestamp>', <hours>, '<format>'?)`  | Add a number of hours to a timestamp.
| `addMinutes('<timestamp>', <minutes>, '<format>'?)`  | Add a number of minutes to a timestamp.
| `addSeconds('<timestamp>', <seconds>, '<format>'?)`  | Add a number of seconds to a timestamp.
| `addToTime('<timestamp>', <interval>, '<timeUnit>', '<format>'?)`   | Add a number of time units to a timestamp.
| `convertFromUtc('<timestamp>', '<destinationTimeZone>', '<format>'?)`  | Convert a timestamp from Universal Time Coordinated (UTC) to the target time zone.
| `convertTimeZone('<timestamp>', '<sourceTimeZone>', '<destinationTimeZone>', '<format>'?)` | Convert a timestamp from the source time zone to the target time zone.
| `convertToUtc('<timestamp>', '<sourceTimeZone>', '<format>'?)` | Convert a timestamp from the source time zone to Universal Time Coordinated (UTC).
| `dayOfMonth('<timestamp>')`   | Return the day of the month component from a timestamp.
| `dayOfWeek('<timestamp>')`    | Return the day of the week from the specified timestamp where Sunday is 0, Monday is 1, and so on.
| `dayOfYear('<timestamp>')`    | Return the day of the year component from a timestamp.
| `formatDateTime('<timestamp>', '<format>'?)` | Return a timestamp in the specified format.
| `startOfDay('<timestamp>', '<format>'?)`     | Return the start of the day for a timestamp. The specified timestamp but starting at the zero-hour mark for the day.
| `startOfHour('<timestamp>', '<format>'?)`   | Return the start of the hour for a timestamp. The specified timestamp but starting at the zero-minute mark for the hour.
| `startOfMonth('<timestamp>', '<format>'?)`  | Return the start of the month for a timestamp. The specified timestamp but starting on the first day of the month at the zero-hour mark.
| `subtractFromTime('<timestamp>', <interval>, '<timeUnit>', '<format>'?)` | Subtract a number of time units from a timestamp.
| `utcNow('<format>'?)` | Return the current timestamp as a string.

See below table for parameter type description.

| Parameter | Type | Description |
| :----------- | :----------- | :----------- |
|`<timestamp>` | String | The string that contains the timestamp. |
|`<interval>` | Integer | The number of specified time units to add |
|`<timeUnit>`| String | The unit of time to use with interval: "Second", "Minute", "Hour", "Day", "Week", "Month", "Year". |
|`<days>`| Integer | The positive or negative number of days to add. |
|`<hours>`| Integer | The positive or negative number of hours to add. |
|`<minutes>`| Integer | The positive or negative number of minutes to add. |
|`<seconds>`| Integer | The positive or negative number of seconds to add.|
|`<destinationTimeZone>`| String |The name for the target time zone. For time zone names, please review: [Microsoft Windows Default Time Zones](https://docs.microsoft.com/en-us/windows-hardware/manufacture/desktop/default-time-zones?view=windows-11). |
|`<sourceTimeZone>`|String |The name for the source time zone. For time zone names, see [Microsoft Windows Default Time Zones](https://docs.microsoft.com/en-us/windows-hardware/manufacture/desktop/default-time-zones?view=windows-11), but you might have to remove any punctuation from the time zone name. |
|`<format>`| String | Either a [single format specifier](https://docs.microsoft.com/en-us/dotnet/standard/base-types/standard-date-and-time-format-strings) or a [custom format pattern](https://docs.microsoft.com/en-us/dotnet/standard/base-types/custom-date-and-time-format-strings). The default format for the timestamp is ["o"](https://docs.microsoft.com/en-us/dotnet/standard/base-types/standard-date-and-time-format-strings) (yyyy-MM-ddTHH:mm:ss.fffffffK), which complies with [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) and preserves time zone information. |

## Set up a schedule with pipeline parameter expressions

### Prerequisites
> Note: Please get below 3 items prepared before running the code blocks.

- Prepare a workspace
    A [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) this schedule will belong to.  
  
- Prepare a published pipeline.
    A published pipeline which will be scheduled. See [how to publish pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-pipelines#publish-a-pipeline).  
      
- Prepare a published pipeline endpoint
    A published pipeline endpoint which will be scheduled. See [how to publish a pipeline to pipeline endpoint](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline_endpoint.pipelineendpoint?view=azure-ml-py#remarks).  
      
    

In [ ]:
from azureml.core import Workspace
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.pipeline.core import PublishedPipeline, PipelineEndpoint

# Get workspace, it need to have at least one published pipeline. 
config_path = "config.json"
ws = Workspace.from_config(path=config_path)

# Get published pipeline id.
all_pub_pipelines = PublishedPipeline.list(ws)
print("Published pipelines found in the workspace: ")
for pub_pipeline in all_pub_pipelines:
    print(pub_pipeline.id)
pub_pipeline_id = all_pub_pipelines[0].id
print("Published pipeline id to be used for schedule operations: {}".format(pub_pipeline_id))

# Get published pipeline endpoint id.
pipeline_endpoint_name = "PipelineEndpointName" # The published pipeline endpoint's name. 
pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name=pipeline_endpoint_name)
pipeline_endpoint_id = pipeline_endpoint.id
print("Published pipeline endpoint id to be used for schedule operations: {}".format(pipeline_endpoint_id))

### Schedule a published pipeline with pipeline parameter expressions

In [ ]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.pipeline.core import PublishedPipeline

# define pipeline parameters
pipeline_parameters = {
    "scheduled_time": "@trigger().scheduledTime",
    "start_time": "@trigger().startTime",
    "utc_now": "@utcNow()",
    "yesterday": "@formatDateTime(subtractFromTime(utcNow(),1,'Day'),'yyyy-MM-dd')"
}

#### Create a schedule with pipeline parameter expressions for published pipeline

In [ ]:
recurrence = ScheduleRecurrence(frequency="Minute", interval=15)
pipeline_schedule = Schedule.create(
    ws, 
    name="ScheduleWithExpression",
    description="Based on time, with pipeline param expressions",
    pipeline_id=pub_pipeline_id,
    pipeline_parameters=pipeline_parameters,
    experiment_name="sample-schedule-with-expression",
    recurrence=recurrence)

# you could search for the schedule triggered pipeline run based on experiment name and description.
pipeline_schedule

#### Update a schedule with pipeline parameter expressions for published pipeline

In [ ]:
update_pipeline_parameters = {
    "start_time": "@addMinutes(trigger().scheduledTime, 10)",
    "user": "@createdBy()",
    "run_id": "@runId()"
}

pipeline_schedule.update(
    name="UpdatedScheduleWithExpression",
    description="Based on time, update pipeline with parameter expressions",
    pipeline_parameters=update_pipeline_parameters)

# you could search for the schedule triggered pipeline run based on experiment name and description.
pipeline_schedule

#### Disable the schedule 

In [ ]:
pipeline_schedule.disable()

### Schedule a published pipeline endpoint with pipeline parameter expressions

In [ ]:
# define pipeline parameters
pipeline_parameters = {
    "dayOfMonth": "@dayOfMonth('2022-01-01 08:00:00')",
    "dayOfWeek": "@dayOfWeek('2022-01-01 08:00:00')",
}

#### Create a schedule with pipeline parameter expressions for published pipeline endpoint

In [ ]:
pipeline_endpoint_schedule = Schedule.create_for_pipeline_endpoint(
    ws,
    name="EndpointScheduleWithExpression01",
    pipeline_endpoint_id=pipeline_endpoint_id,
    experiment_name="sample-endpoint-schedule-with-expression",
    description="Create for pipeline endpoint with pipeline parameter expressions",
    pipeline_parameters=pipeline_parameters,
    recurrence=recurrence)

# you could search for the schedule triggered pipeline run based on experiment name and description.
pipeline_endpoint_schedule

#### Update a schedule with pipeline parameter expressions for published pipeline endpoint  
> Note: If you want to update pipeline parameter expression of published pipeline endpoint, you need to update the previous schedule in below way, as simply republishing the pipeline will not work.

In [ ]:
updated_recurrence = ScheduleRecurrence(frequency="Minute", interval=5)
updated_pipeline_parameters = {
    "dayOfMonth": "@dayOfMonth(utcNow())",
    "dayOfWeek": "@dayOfWeek(utcNow())",
}

pipeline_endpoint_schedule.update(
    name="UpdateEndpointScheduleWithExpression",
    description="Based on time, update pipeline endpoint with parameter expressions",
    pipeline_parameters=updated_pipeline_parameters,
    recurrence=updated_recurrence)

# you could search for the schedule triggered pipeline run based on experiment name and description.
pipeline_endpoint_schedule

#### Disable the schedule 

In [ ]:
pipeline_endpoint_schedule.disable()

## Questions and Answers

Q1: Why are my input pipeline parameter expressions not resolved?

A1: Pipeline parameter expressions are case sensitive. Please check if you're using supported built-in functions in input expression, otherwise used as raw string.


Q2: What should I do when I meet error "Invalid pipeline parameter expression"?

A2: If you want to use resolved value of the expression, please check if all the functions in the expression have been set with correct type parameters.


Q3: Why do new triggered runs still use previous pipeline parameter expressions although I have republished with new expressions?

A3: If you want to update pipeline parameter expressions, please update schedule with new ones. Republishing pipeline only works for updating pipeline parameter with fixed value.
